In [170]:
from officelib.xllib import *
from officelib.const import xlconst as xlc
from pywintypes import com_error
xl = Excel()
w,i = None, None
for i, w in enumerate(xl.Workbooks):
    print(i, w.Name)
del w

0 gc 500 nonsense.xlsx


In [158]:
fp = "C:/users/nathan/downloads/gc 500 nonsense.xlsx"
wb = xl.Workbooks.Open(fp)

In [159]:
ws = wb.Worksheets(1)
cells = ws.Cells
cr = cells.Range

In [160]:
charts = []

LEFT = 5230

def drange(c):
    x1 = c.Offset(2,1)
    x2 = x1.End(xlc.xlDown)
    y1 = x1.Offset(1,2)
    y2 = x2.Offset(1,2)
    return x1, x2, y1, y2
    
def drangen(name):
    c = cells.Find(name)
    if c is not None:
        return drange(c)
    else:
        raise ValueError(name)
        
def data(n):
    x1, x2, y1, y2 = drangen(n)
    return list(zip(*cr(x1, y2).Value))

def mkchart(c, name="", i=0):
    x1, x2, y1, y2 = drange(c)
    xd = cr(x1,x2)
    yd = cr(y1,y2)
    ch = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    
    FormatChart(ch, None, name, "Date", Legend=False)
    
    co = ch.Parent  # ChartObject
    co.Left = LEFT
    co.Height = 300
    co.Top  = 50 + (co.Height+20) * i
    co.Width = 500
    
    CreateDataSeries(ch, xd, yd, name)
    return ch

def chartv(name, i=0):
    c = cells.Find(name)
    if c is not None:
        return mkchart(c, name, i)
    else:
        raise NameError(name)
        
co = None
for co in ws.ChartObjects():
    co.Delete()
del co
        
vars = (
        "TempHeatDutyActual(%)", 
        "TempPV(C)", 
    #    "TempModeActual", 
        "TempSP(C)"
    )
for i, v in enumerate(vars):
    ch = chartv(v, i)
    charts.append(ch)
    


In [161]:
import datetime
EPOCH = datetime.datetime(1900, 1, 1)
def xl2dt(n):
    return EPOCH + datetime.timedelta(days=n-2)

def dt2xl(dt):
    return (dt - EPOCH).total_seconds()/(60*60*24) + 2

In [162]:
def setxax(xmin, xmax):
    x1 = dt2xl(xmin)
    x2 = dt2xl(xmax)
    for ch in charts:
        FormatAxesScale(ch, x1, x2)

In [163]:
def date(s):
    fmts = (
        "%m/%d/%y %H:%M:%S",
        "%m/%d/%y %I:%M:%S %p",
        "%m/%d/%y %I:%M %p",
        "%m/%d/%y %H:%M",
        "%m/%d/%Y %H:%M:%S",
        "%m/%d/%Y %H:%M",
        "%m/%d/%Y %I:%M:%S %p",
        "%m/%d/%Y %I:%M %p",
    )
    for f in fmts:
        try:
            return datetime.datetime.strptime(s,f)
        except ValueError:
            pass
    raise ValueError("%r does not match a date format"%s)

In [164]:
def charttype(t):
    for ch in charts:
        ch.ChartType = t

In [165]:
d1 = date("12/18/2017  11:06:54 AM")
d2 = date("12/18/2017  11:17:10 AM")

In [155]:
setxax(d1, d2)

In [75]:
types = [xlc.xlXYScatter, xlc.xlXYScatterLinesNoMarkers]
charttype(types[i])
i = 1-i

In [76]:
i = 0
import time
while True:
    try:
        charttype(types[i])
        i = 1-i
        time.sleep(1)
    except KeyboardInterrupt:
        break

In [125]:
REFDT = datetime.datetime(2017, 12, 18).timestamp()

def xl2mpl(dt):
    return (dt.timestamp()-REFDT)/60

In [113]:
%matplotlib
import matplotlib.pyplot as plt
import numpy as np

Using matplotlib backend: TkAgg


In [114]:
def mpd(d):
    return np.array(d)


In [121]:
plt.close('all')
for v in vars:
    x,y = data(v)
    x = map(xl2mpl, x)
    x = mpd(list(x))
    y = mpd(y)
    f = plt.figure()
    ax = f.add_subplot(111)
    ax.plot(x,y)
    ax.set_title(v)

In [144]:
plt.close('all')
def mkfig(v):
    x,y = data(v)
    x = map(xl2mpl, x)
    x = mpd(list(x))
    y = mpd(y)
    f = plt.figure()
    ax = f.add_subplot(111)
    ax.plot(x,y)
    ax.set_title(v)
    return ax
ax = mkfig("TempHeatDutyActual(%)")
xmode,ymode = data("TempModeActual")
xmin = 180
xmax = 200
ax.set_xlim(xmin, xmax)

(180, 200)

In [145]:
t = ymode[0]
i = 0
l = len(xmode)
while i < l:
    y = ymode[i]
    x = xl2mpl(xmode[i])
    #if y != t:
    if y == 0:
        ax.axvline(x, 0, 100, ls="--", color="orange", linewidth=1)
    elif y == 2:
        #ax.axvline(x, 0, 100, ls="--", color="blue", linewidth=1)
        t = y
    i = i + 1

In [146]:
xsp, ysp = data("TempSP(C)")
i = 0
l = len(xsp)
while i < l:
    y = ysp[i]
    x = xl2mpl(xsp[i])
    ax.plot(x,y, "+", mfc=None, mec="r", mew=2, ms=8)
    i += 1

In [180]:
class XLCtx():
    def __init__(self, xl, wb):
        self.xl = xl
        if isinstance(wb, str):
            # active?
            try:
                wb = xl.Workbooks(wb)
            except com_error:
                try:
                    wb = xl.Workbooks.Open(wb)
                except com_error:
                    raise FileNotFoundError(wb) from None
        self.wb = wb
        self.ws = wb.Worksheets(1)
        self.cells = self.ws.Cells
        self.cr = self.cells.Range
        
    @classmethod
    def from_recent_file(cls, xl, fn):
        for rf in xl.RecentFiles:
            if rf.Name == fn:
                break
        else:
            raise FileNotFoundError(fn)
        return cls(xl, rf.Path)
    from_recent = from_rf = from_recent_file

In [181]:
ue = XLCtx.from_recent_file(xl, "gc user events.xlsx")

In [185]:
data = ue.cr(ue.cr("A2"), ue.cr("A2").End(xlc.xlDown).Offset(1,2)).Value
data2 = cr("")

In [194]:
start = data[0][1]
fmt = "%m/%d %H:%M:%S"
for e, t in data:
    if e.startswith("temp sensor"):
        a,b,c = e.split(";")
        mode = b.split()[0]
        dt = (t-start).total_seconds()/60
        if mode.lower() == "auto":
            sp = c.split()[-1].rstrip(".")
            sp = float(sp)
            print(t.strftime(fmt), "Temp %s: %.1f"%(mode, sp))
        else:
            print(t.strftime(fmt), "Temp %s"%mode)

12/18 11:04:49 Temp Auto: 30.0
12/18 11:17:09 Temp Off
12/18 12:49:52 Temp Auto: 0.0
12/18 12:52:11 Temp Off
01/02 15:31:53 Temp Off
01/02 15:36:44 Temp Auto: 37.0
01/04 10:02:58 Temp Off
01/04 10:29:48 Temp Off


In [208]:
def l2v(l):
    return l/20*5

In [212]:
start = data[0][1]
fmt = "%m/%d %H:%M:%S"
for e, t in data:
    if e.startswith("agitation sensor"):
        a,b,c = e.split(";")
        mode = b.split()[0]
        dt = (t-start).total_seconds()/60
        if mode.lower() in ("auto", "man"):
            sp = c.split()[-1].rstrip(".")
            sp = float(sp)
            sps = "%.1f"%sp
            lvs = "%-.1f"%(l2v(sp))
            lvs2 = str(l2v(sp) > 0.3)
        else:
            sps = ""
            lvs = ""
            lvs2 = "False"
        print("%-15s %s %-8s %-10s %10s %10s" %(t.strftime(fmt), "Agitation", mode+":", sps, lvs, lvs2))

12/18 11:04:43  Agitation Auto:    25.0              6.2       True
12/18 11:06:48  Agitation Man:     1.1               0.3      False
12/18 11:07:01  Agitation Auto:    25.0              6.2       True
12/18 11:07:14  Agitation Man:     1.1               0.3      False
12/18 11:07:24  Agitation Man:     2.0               0.5       True
12/18 11:07:36  Agitation Man:     3.0               0.8       True
12/18 11:07:45  Agitation Man:     4.0               1.0       True
12/18 11:07:55  Agitation Man:     5.0               1.2       True
12/18 11:08:11  Agitation Man:     7.0               1.8       True
12/18 11:08:25  Agitation Man:     10.0              2.5       True
12/18 11:08:51  Agitation Off:                                False
12/18 11:08:56  Agitation Off:                                False
12/18 11:09:46  Agitation Auto:    25.0              6.2       True
12/18 11:10:40  Agitation Auto:    25.0              6.2       True
12/18 11:10:50  Agitation Man:     15.0         